## 引入xlwings
xlwings可以方便和现有excel表格进行交互。

In [1]:
import xlwings as xw
import pandas as pd

## 打开一个excel
如果你希望打开目标excel文件，则Book接受目标excel文件的路径参数。

In [2]:
book = xw.Book()
book.name

'工作簿1'

## 获得目标excel的sheet对象
如果目标sheet有其他名字，则可以 `book.sheets['更有含义的名字']` 来调用，后面的很多操作都在sheet对象下进行。

In [3]:
book.sheets

Sheets([<Sheet [工作簿1]Sheet1>])

In [4]:
sheet1 = book.sheets[0]

## 调用某个range
如果你不清楚excel的range概念，请参阅《Excel 2019 Bible》一书。

In [5]:
sheet1.range("A1")

<Range [工作簿1]Sheet1!$A$1>

## 对某个range写入值
如果你熟悉range的概念，那么就不会对代码执行结果感到意外，有点类似于粘贴动作。

In [6]:
sheet1.range("A1").value = [[1, 2],
                            [3, 4]]
sheet1.range("A4").value = "Hello!"

## 获得某个range的值

In [7]:
sheet1.range("A1:B2").value

[[1.0, 2.0], [3.0, 4.0]]

In [8]:
sheet1.range("A1:B2")[0, 0]

<Range [工作簿1]Sheet1!$A$1>

In [9]:
sheet1.range("A1:B2")[:, 1]

<Range [工作簿1]Sheet1!$B$1:$B$2>

In [10]:
sheet1["A1"]

<Range [工作簿1]Sheet1!$A$1>

In [11]:
sheet1["A1:B2"]

<Range [工作簿1]Sheet1!$A$1:$B$2>

In [12]:
sheet1[0, 0]

<Range [工作簿1]Sheet1!$A$1>

In [13]:
sheet1[:2, :2]

<Range [工作簿1]Sheet1!$A$1:$B$2>

## pandas的DataFrame对象写入excel
你可以通过 `header` 和 `index` 参数来控制DataFrame的写入行为。

In [14]:
data = [["Mark", 55, "Italy", 4.5, "Europe"],
        ["John", 33, "USA", 6.7, "America"]]
df = pd.DataFrame(data=data,
                  columns=["name", "age", "country",
                           "score", "continent"],
                  index=[1001, 1000])
df.index.name = "user_id"
df

,name,age,country,score,continent
user_id,,,,,
1001,Mark,55,Italy,4.5,Europe
1000,John,33,USA,6.7,America


In [15]:
sheet1["A6"].value = df

In [16]:
sheet1["B10"].options(header=False, index=False).value = df

## excel某个range内的数据转为DataFrame对象
expand方法会自动扩展为相连一片的range，options接受 `pd.DataFrame` 表示接受的值会转成DataFrame对象。

你可以通过 `header` 和 `index` 参数来控制转出行为。

In [17]:
df2 = sheet1["A6"].expand().options(pd.DataFrame).value
df2

,name,age,country,score,continent
user_id,,,,,
1001.0,Mark,55.0,Italy,4.5,Europe
1000.0,John,33.0,USA,6.7,America


In [18]:
df2.index = df2.index.astype(int)
df2

,name,age,country,score,continent
user_id,,,,,
1001,Mark,55.0,Italy,4.5,Europe
1000,John,33.0,USA,6.7,America


In [19]:
sheet1["A6"].expand().options(pd.DataFrame, index=False).value

,user_id,name,age,country,score,continent
0,1001.0,Mark,55.0,Italy,4.5,Europe
1,1000.0,John,33.0,USA,6.7,America


In [20]:
df3 = sheet1["B10:F11"].options(pd.DataFrame, header=False, index=False).value
df3

,0,1,2,3,4
0,Mark,55.0,Italy,4.5,Europe
1,John,33.0,USA,6.7,America


## 插入excel图表

In [21]:
sheet1["A15"].value = [[None, "North", "South"],
                       ["Last Year", 2, 5],
                       ["This Year", 3, 6]]

In [22]:
chart = sheet1.charts.add(top=sheet1["A19"].top,
                          left=sheet1["A19"].left)
chart.chart_type = "column_clustered"
chart.set_source_data(sheet1["A15"].expand())

In [23]:
df4 = sheet1["A15"].expand().options(pd.DataFrame).value
df4

,North,South
Last Year,2.0,5.0
This Year,3.0,6.0


## 插入excel全局常数

In [24]:
book.names.add("Var_1", 1)
book.names

[<Name 'Var_1': =1>]

## 删除某个excel全局常数

In [25]:
if 'Var_1' in book.names:
    book.names['Var_1'].delete()
    print('deleted')

book.names

deleted


[]

## 保存文档
如果打开的是目标excel文件，那么调用 `book.save()` 即可。

In [26]:
book.save('temp.xlsx')